# Telemetry Pipeline - Model Compression Workflow
This notebook takes the distilled_pruned_gpt2_telemetry model, performs model compression using quantization.

                    GNU AFFERO GENERAL PUBLIC LICENSE
                       Version 3, 19 November 2007

Copyright (C) 2025 Shaji R. Nathan  
IP Infusion Inc.  
Email: shaji.nathan@ipinfusion.com  

This program is free software: you can redistribute it and/or modify  
it under the terms of the GNU Affero General Public License as  
published by the Free Software Foundation, either version 3 of the  
License, or (at your option) any later version.  

This program is distributed in the hope that it will be useful,  
but WITHOUT ANY WARRANTY; without even the implied warranty of  
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the  
GNU Affero General Public License for more details.  

You should have received a copy of the GNU Affero General Public License  
along with this program. If not, see <https://www.gnu.org/licenses/>.  

As per AGPLv3, if you modify this software and make it available over a  
network, you must provide the source code of your modifications under the  
same license.  

For inquiries, please contact:  
Shaji R. Nathan  
IP Infusion Inc.  
Email: shaji.nathan@ipinfusion.com  


## Check Distilled and Pruned Model for corruption 

In [ ]:
import torch, torchvision
from transformers import AutoModelForCausalLM, AutoTokenizer
torchvision.disable_beta_transforms_warning()

In [ ]:

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"PyTorch Version: {torch.__version__}")

try:
    teacher = AutoModelForCausalLM.from_pretrained("./distilled_pruned_gpt2_telemetry", device_map=None)
    print("✅ Model loaded successfully to CPU.")

    teacher = teacher.to("cuda")
    print("✅ Model moved to GPU successfully.")
except Exception as e:
    print(f"❌ Error during model load/move: {e}")

# Quantization of the model 
### Load and Quantize  GPT-2 Model in 4-bit

In [1]:
# Install required libraries (uncomment if needed)
# !pip install auto-gptq transformers optimum torch

import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

# Step 1: Load Tokenizer
print("🔹 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("gpt2")  # Load the base tokenizer

# Fix: Add padding token (GPT-2 does not have one by default)
tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding

# Step 2: Define the 4-bit Quantization Configuration
print("🔹 Defining 4-bit quantization configuration...")
quantize_config = BaseQuantizeConfig(
    bits=4,  # Use 4-bit quantization
    group_size=128,  # Grouped quantization for better efficiency
    desc_act=False  # Disable activation-descending quantization
)

# Step 3: Load the Pruned and Distilled GPT-2 Model
print("🔹 Loading the model...")
model = AutoGPTQForCausalLM.from_pretrained(
    "./distilled_pruned_gpt2_telemetry",  # Path to the fine-tuned model
    quantize_config=quantize_config  # Apply quantization settings
)

# Step 4: Prepare Example Sentences (DO NOT Tokenize Here)
print("🔹 Preparing example sentences for quantization...")
example_sentences = [
    "Hello, how are you?",
    "The future of AI is promising.",
    "Quantization reduces memory usage while maintaining performance.",
    "GPT-2 can generate human-like text responses."
]

# Step 5: Tokenize Sentences and Convert Format (FIXED)
print("🔹 Tokenizing and formatting input for quantization...")

# Tokenize sentences and extract `input_ids` and `attention_mask`
tokenized_examples = tokenizer(
    example_sentences,
    padding=True,  # Ensure padding
    truncation=True,  # Truncate long inputs
    return_tensors="pt"  # Return PyTorch tensors
)

# Convert tokenized output into a list of dicts (Required Format)
examples_for_quantization = [
    {"input_ids": input_id, "attention_mask": attention_mask}
    for input_id, attention_mask in zip(tokenized_examples["input_ids"], tokenized_examples["attention_mask"])
]

# Step 6: Quantize the Model Using Tokenized Examples (FIXED)
print("🔹 Quantizing the model...")
model.quantize(examples_for_quantization)  # ✅ Now it will work!

# Step 7: Move Model to CPU (or GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"✅ Model successfully quantized to 4-bit and running on {device}!")

# Step 8: Save the Quantized Model Locally
save_directory = "./quantized_distilled_pruned_gpt2_telemetry"
print(f"🔹 Saving the quantized model to {save_directory}...")
model.save_quantized(save_directory)  # ✅ Now this will work
tokenizer.save_pretrained(save_directory)  # Save tokenizer

print(f"✅ Quantized model and tokenizer saved successfully in {save_directory}!")

# Step 9: Verify Saved Files
import os
print("\n📁 Saved files:")
print(os.listdir(save_directory))

# Step 10: Reload the Saved Quantized Model
print("\n🔹 Reloading the quantized model for verification...")
reloaded_model = AutoGPTQForCausalLM.from_quantized(save_directory)
reloaded_model.to(device)
print("✅ Quantized model reloaded successfully!")

# Step 11: Run Inference with the Quantized Model
print("\n🔹 Running inference with the quantized model...")
prompt = "The future of AI is"
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)  # Ensure padding

# Generate text
with torch.no_grad():
    outputs = reloaded_model.generate(**inputs, max_length=50)

# Decode and print output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n📝 Generated Text:")
print(generated_text)


C:\Miniconda3\envs\sentence-transformers\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Miniconda3\envs\sentence-transformers\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedba

🔹 Loading tokenizer...
🔹 Defining 4-bit quantization configuration...
🔹 Loading the model...


Some weights of the model checkpoint at ./distilled_pruned_gpt2_telemetry were not used when initializing GPT2LMHeadModel: ['lm_head.weight_mask']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


🔹 Preparing example sentences for quantization...
🔹 Tokenizing and formatting input for quantization...
🔹 Quantizing the model...


INFO - Start quantizing layer 1/6
INFO - Quantizing attn.c_attn in layer 1/6...
INFO - Quantizing attn.c_proj in layer 1/6...
INFO - Quantizing mlp.c_fc in layer 1/6...
INFO - Quantizing mlp.c_proj in layer 1/6...
INFO - Start quantizing layer 2/6
INFO - Quantizing attn.c_attn in layer 2/6...
INFO - Quantizing attn.c_proj in layer 2/6...
INFO - Quantizing mlp.c_fc in layer 2/6...
INFO - Quantizing mlp.c_proj in layer 2/6...
INFO - Start quantizing layer 3/6
INFO - Quantizing attn.c_attn in layer 3/6...
INFO - Quantizing attn.c_proj in layer 3/6...
INFO - Quantizing mlp.c_fc in layer 3/6...
INFO - Quantizing mlp.c_proj in layer 3/6...
INFO - Start quantizing layer 4/6
INFO - Quantizing attn.c_attn in layer 4/6...
INFO - Quantizing attn.c_proj in layer 4/6...
INFO - Quantizing mlp.c_fc in layer 4/6...
INFO - Quantizing mlp.c_proj in layer 4/6...
INFO - Start quantizing layer 5/6
INFO - Quantizing attn.c_attn in layer 5/6...
INFO - Quantizing attn.c_proj in layer 5/6...
INFO - Quantizing 

...

✅ Quantized model and tokenizer saved successfully in ./quantized_distilled_pruned_gpt2_telemetry!

📁 Saved files:
['config.json', 'gptq_model-4bit-128g.safetensors', 'merges.txt', 'quantize_config.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.json']

🔹 Reloading the quantized model for verification...


INFO - The layer lm_head is not quantized.


✅ Quantized model reloaded successfully!

🔹 Running inference with the quantized model...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📝 Generated Text:
The future of AI is in question.


- Timestamp: 2025-03-07 09:21:21.737018
- Module: QSFP-13
- Temperature: 66.37°C
- Trans
